# PyMC Version - Much Easier!

This is the new PyMC-based implementation. **No more Stan errors!**

## Why PyMC is Better:
- ✓ Pure Python (easy debugging)
- ✓ Clear error messages
- ✓ No compilation step
- ✓ Interactive development
- ✓ Built-in diagnostics

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pydmc import WaldStopSignalModel

np.random.seed(42)

## Generate Test Data

In [ ]:
# Simple synthetic data
data = []
for subj in ['S01', 'S02', 'S03']:
    for trial in range(100):
        stimulus = np.random.choice([0, 1])
        # Mostly correct responses
        if np.random.rand() < 0.9:
            response = stimulus + 1
        else:
            response = 2 - stimulus
        
        # Realistic RTs
        rt = np.random.gamma(3, 0.1) + 0.3
        
        data.append({
            'subject': subj,
            'stimulus': stimulus,
            'response': response,
            'rt': rt,
            'ssd': np.nan  # All go trials for simplicity
        })

data = pd.DataFrame(data)
print(f"Created {len(data)} trials from {data['subject'].nunique()} subjects")
data.head()

## Fit the Model (It's Easy!)

In [ ]:
# Create model
model = WaldStopSignalModel(use_hierarchical=True)

# Fit - notice how simple!
trace = model.fit(
    data,
    draws=500,      # Number of samples
    tune=500,       # Tuning/warmup
    chains=2,       # Number of chains
    cores=2         # Parallel chains
)

## View Results

In [ ]:
# Summary statistics
summary = model.summary()

## Plot Traces (Check Convergence)

In [ ]:
# Trace plots - should look like "hairy caterpillars"
model.plot_traces()

## Plot Posterior Distributions

In [ ]:
# Posterior distributions
model.plot_posterior(var_names=['mu_B', 'mu_t0', 'mu_v_true', 'mu_v_false'])

## Save Results

In [ ]:
# Save to file (NetCDF format)
model.save_results('../models/pymc_results.nc')

## Load Results Later

In [ ]:
# Can load results in a new session
# model.load_results('../models/pymc_results.nc')
# model.summary()

## Debugging is Easy!

If something goes wrong, you get **clear Python errors** instead of cryptic Stan messages:

In [ ]:
# Example: You can inspect the model directly
print("Model structure:")
print(model.model)

# You can even step through the code with a debugger!
# Just add: import pdb; pdb.set_trace()

## Comparison: Stan vs PyMC

### Stan (Old):
```python
# Opaque errors
RuntimeError: Error during sampling
Log probability evaluates to log(0)
# Where? What parameter? Who knows!

# Need to:
- Wait for compilation (slow!)
- Parse cryptic C++ errors
- Use show_console=True
- Debug blind
```

### PyMC (New):
```python
# Clear Python errors
ValueError: Wald parameter mu must be positive, got -0.5
  File "models.py", line 187, in build_model
    mu_wald = B_trial / v_trial
# Exactly where! What value! Easy to fix!

# You get:
- No compilation
- Clear stack traces
- Can use Python debugger
- Inspect everything
```

**Much better!**